# Code generator and executor

Apply group chat conversation arquitecture between two agents, a code generator and a code executor, to generate a graph of closing prices of NVIDIA for the past month.

### Setup

In [25]:
# Import all necessary packages
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.code_executors.jupyter import JupyterCodeExecutor
from autogen_core.code_executor import CodeBlock
from autogen_core import CancellationToken
from dotenv import load_dotenv
import os

In [26]:
# Load .env file
load_dotenv(override=True)

# Get environment variables
openai_key = os.getenv("OPENAI_KEY")
openai_endpoint = os.getenv("OPENAI_ENDPOINT")

In [27]:
# Create Azure OpenAI Client
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    model="gpt-4o",
    api_version="2024-06-01",
    azure_endpoint=openai_endpoint,
    api_key=openai_key
)

### Define the task

In [ ]:
import datetime

today = datetime.datetime.now().date()
task = f"Today is {today}. "\
"Fetch the closing prices of NVDA for the past month and plot them. "\
"Save the figure as nvda_price_plot.png."\
"Use alpha-vantage python library to access data, this is the necesary API key: C33RNYMS3ANCK3OL"

### Define code execution tools

In [29]:
async def install_dependencies(code_block:str) -> str:
    async with JupyterCodeExecutor() as executor:
        cancel_token = CancellationToken()
        code_blocks = [CodeBlock(code=code_block, language="sh")]
        code_result = await executor.execute_code_blocks(code_blocks, cancel_token)
        print(code_result)

        return code_result
    
async def execute_code(code_block:str) -> str:
    async with JupyterCodeExecutor() as executor:
        cancel_token = CancellationToken()
        code_blocks = [CodeBlock(code=code_block, language="python")]
        code_result = await executor.execute_code_blocks(code_blocks, cancel_token)
        print(code_result)

        return code_result

### Define group of agents

In [30]:
engineer = AssistantAgent(
    name="Engineer",
    description = "An engineer that writes code to acomplish a task.",
    system_message="""You are a coder. You write code to acomplish a task.
    Only write python code and nothing else.
    Put attention to previous errors, rewrite your code avoding the same mistakes.
    Check if CodeExecutor agent has executed the code correctly, if so, return only 'TERMINATE'.""",
    model_client=az_model_client,
)

code_executor = AssistantAgent(
    name="CodeExecutor",
    system_message="""You are a code executor, you execute the code Engineer passes to you.
    
    First, you always check if there is any dependencies on the code the engineer generates, and use install_dependencies tool to install that dependencies.
    For example: '''pip install numpy'''

    Then, use execute_code tool to execute code.
    For example: '''print('Hello world')'''
    """,
    model_client=az_model_client,
    tools=[install_dependencies, execute_code]
)

In [33]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.ui import Console

termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(6)
team = RoundRobinGroupChat(
    [engineer, code_executor],
    termination_condition = termination,
)

### Run team of agents

In [34]:
# `run_stream` returns an async generator to stream the intermediate messages.
stream = team.run_stream(task=task)
# `Console` is a simple UI to display the stream.
await Console(stream)

---------- user ----------


Today is 2025-02-21. Fetch the closing prices of NVDA for the past month and plot them. Save the figure as nvda_price_plot.png.Use alpha-vantage python library to access data, this is the necesary API key: C33RNYMS3ANCK3OL
---------- Engineer ----------
```python
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

# Alpha Vantage API key
api_key = 'C33RNYMS3ANCK3OL'

# Initialize TimeSeries with your key
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch the daily time series for NVDA
data, _ = ts.get_daily(symbol='NVDA', outputsize='full')

# Filter data for the past month (assuming today's date is 2025-02-21)
data.index = pd.to_datetime(data.index)
start_date = '2025-01-21'
end_date = '2025-02-21'
data_past_month = data.loc[start_date:end_date]

# Plot the closing prices
plt.figure(figsize=(10, 6))
plt.plot(data_past_month.index, data_past_month['4. close'], label='Closing Price', color='b', alpha=0.7)
plt.title('NVDA Closin

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Today is 2025-02-21. Fetch the closing prices of NVDA for the past month and plot them. Save the figure as nvda_price_plot.png.Use alpha-vantage python library to access data, this is the necesary API key: C33RNYMS3ANCK3OL', type='TextMessage'), TextMessage(source='Engineer', models_usage=RequestUsage(prompt_tokens=769, completion_tokens=258), content="```python\nimport matplotlib.pyplot as plt\nfrom alpha_vantage.timeseries import TimeSeries\nimport pandas as pd\n\n# Alpha Vantage API key\napi_key = 'C33RNYMS3ANCK3OL'\n\n# Initialize TimeSeries with your key\nts = TimeSeries(key=api_key, output_format='pandas')\n\n# Fetch the daily time series for NVDA\ndata, _ = ts.get_daily(symbol='NVDA', outputsize='full')\n\n# Filter data for the past month (assuming today's date is 2025-02-21)\ndata.index = pd.to_datetime(data.index)\nstart_date = '2025-01-21'\nend_date = '2025-02-21'\ndata_past_month = data.loc[start_dat